In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('mouse_tracking_data.txt')

df

,event_type,button,x,y,delta,time
0,move,NaN,854.0,162.0,NaN,1.694859e+09
1,move,NaN,856.0,164.0,NaN,1.694859e+09
2,move,NaN,863.0,167.0,NaN,1.694859e+09
3,move,NaN,874.0,172.0,NaN,1.694859e+09
4,move,NaN,895.0,185.0,NaN,1.694859e+09
...,...,...,...,...,...,...
585,move,NaN,926.0,255.0,NaN,1.694859e+09
586,move,NaN,919.0,255.0,NaN,1.694859e+09
587,move,NaN,919.0,255.0,NaN,1.694859e+09
588,move,NaN,918.0,256.0,NaN,1.694859e+09
